# Lab 4
# Extend ETL pipeline 

#### Imports

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os 

#### Adding more rows to the DataFrame


In [20]:
import pandas as pd
import random
from datetime import datetime, timedelta

OUTPUT_FILENAME = 'custom_data.csv'
NUM_NEW_ROWS = 30
SALES_PER_DAY_RANGE = (3, 8) 

CUSTOMERS = ['Amazon', 'Walmart', 'Target', 'Costco', 'BestBuy', 'eBay', 'Microsoft', 'Google', 'Apple', 'Meta']
PRODUCT_CATEGORIES = ['Electronics', 'Home Goods', 'Apparel', 'Books', 'Groceries', 'Software', 'Tools', 'Sports']
PAYMENT_METHODS = ['Credit Card', 'Debit Card', 'PayPal', 'Bank Transfer']
STATUS_OPTIONS = ['Completed', 'Pending', 'Cancelled', 'Refunded'] 

print(f"Attempting to add {NUM_NEW_ROWS} new records to '{OUTPUT_FILENAME}'...")

try:
    existing_df = pd.read_csv(OUTPUT_FILENAME)
    if not existing_df.empty:
        record_id_counter = existing_df['record_id'].max() + 1
    else:
        record_id_counter = 1
except FileNotFoundError:
    print(f"'{OUTPUT_FILENAME}' not found. Creating a new file with new records.")
    existing_df = pd.DataFrame()
    record_id_counter = 1 

START_DATE_FOR_NEW_DATA = datetime(2025, 6, 1)

new_records = []
num_records_generated = 0
current_date_for_new_data = START_DATE_FOR_NEW_DATA

while num_records_generated < NUM_NEW_ROWS:
  
    if num_records_generated > 0: 
        current_date_for_new_data += timedelta(days=1)

    transaction_time = current_date_for_new_data + timedelta(
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59)
    )

    last_updated_time = transaction_time + timedelta(
        minutes=random.randint(1, 120) 
    )
  
    if last_updated_time.date() > current_date_for_new_data.date():
        last_updated_time = current_date_for_new_data.replace(hour=23, minute=59, second=59)

    new_records.append({
        'record_id': record_id_counter,
        'transaction_date': current_date_for_new_data.date().isoformat(),
        'transaction_timestamp': transaction_time.isoformat(),
        'customer_name': random.choice(CUSTOMERS),
        'product_category': random.choice(PRODUCT_CATEGORIES),
        'amount': round(random.uniform(10.00, 2000.00), 2),
        'quantity': random.randint(1, 10),
        'payment_method': random.choice(PAYMENT_METHODS),
        'last_updated_timestamp': last_updated_time.isoformat(),
        'status': random.choice(STATUS_OPTIONS)
    })
    record_id_counter += 1
    num_records_generated += 1

new_df = pd.DataFrame(new_records)
combined_df = pd.concat([existing_df, new_df], ignore_index=True)
combined_df.to_csv(OUTPUT_FILENAME, index=False)

print(f"Successfully added {NUM_NEW_ROWS} new records to '{OUTPUT_FILENAME}'.")

Attempting to add 30 new records to 'custom_data.csv'...
Successfully added 30 new records to 'custom_data.csv'.


#### Loading data

In [21]:
DATA_FILE = 'custom_data.csv' 
LAST_EXTRACTION_FILE = 'last_extraction.txt'
OUTPUT_FULL_TRANSFORMED_FILENAME = 'transformed_full.csv'
OUTPUT_INCREMENTAL_TRANSFORMED_FILENAME = 'transformed_incremental.csv'

#### Full extraction
- loading the full dataset

In [22]:

full_data_df = pd.read_csv(DATA_FILE)
print(full_data_df.head())
print("Shape:", full_data_df.shape)

   record_id transaction_date transaction_timestamp customer_name  \
0          1       2025-04-01   2025-04-01T17:49:57        Costco   
1          2       2025-04-01   2025-04-01T10:36:18         Apple   
2          3       2025-04-01   2025-04-01T17:16:47        Target   
3          4       2025-04-01   2025-04-01T18:15:48        Costco   
4          5       2025-04-01   2025-04-01T02:37:35        Amazon   

  product_category   amount  quantity payment_method last_updated_timestamp  \
0      Electronics   929.94         1         PayPal    2025-04-01T18:49:57   
1        Groceries   130.17         4         PayPal    2025-04-01T12:59:18   
2       Home Goods  1333.76         8    Credit Card    2025-04-01T18:19:47   
3            Tools  1115.54         9         PayPal    2025-04-01T20:41:48   
4         Software   633.14         6    Credit Card    2025-04-01T03:35:35   

      status  
0    Pending  
1  Cancelled  
2    Pending  
3  Cancelled  
4  Cancelled  
Shape: (398, 10)


### Incremental extraction

In [23]:
def get_last_timestamp(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            timestamp_str = f.read().strip()
            if timestamp_str:
                return datetime.fromisoformat(timestamp_str)
    return datetime.min 

last_extraction_time = get_last_timestamp(LAST_EXTRACTION_FILE)
print(f"Last extraction timestamp: {last_extraction_time}")

incremental_data_df = pd.DataFrame() 

try:
    if not full_data_df.empty: 
        full_data_df['last_updated_timestamp'] = pd.to_datetime(full_data_df['last_updated_timestamp'], errors='coerce')
        
        incremental_data_df = full_data_df[
            full_data_df['last_updated_timestamp'] > last_extraction_time
        ].copy() 

        print(f"Extracted {incremental_data_df.shape[0]} incremental rows.")
        if not incremental_data_df.empty:
            print("First 5 rows of incremental data:\n", incremental_data_df.head())
        else:
            print("No new or updated records found.")
    else:
        print("No full data to perform incremental extraction from.")
except Exception as e:
    print(f"An error occurred during incremental extraction: {e}")

Last extraction timestamp: 2025-06-14 11:46:34.700903
Extracted 33 incremental rows.
First 5 rows of incremental data:
      record_id transaction_date transaction_timestamp customer_name  \
352        353       2025-06-15   2025-06-15T04:08:57        Costco   
353        354       2025-06-16   2025-06-16T20:11:05        Costco   
354        355       2025-06-17   2025-06-17T03:46:57        Amazon   
355        356       2025-06-18   2025-06-18T13:03:27        Amazon   
356        357       2025-06-19   2025-06-19T15:37:21        Google   

    product_category   amount  quantity payment_method last_updated_timestamp  \
352            Books  1621.91         5  Bank Transfer    2025-06-15 04:46:57   
353            Books   432.92         2     Debit Card    2025-06-16 20:47:05   
354            Tools   876.54         7     Debit Card    2025-06-17 05:45:57   
355            Books  1435.31         5    Credit Card    2025-06-18 13:36:27   
356       Home Goods    39.00         3  Bank Tr

## Transformations
- cleaning
- standradization
- type conversion

In [24]:
def apply_transformations(df):
    transformed_df = df.copy()

    # Removing duplicates
    transformed_df.drop_duplicates(subset=['record_id'], keep='first', inplace=True)

    # Stripping whitespace from string columns
    for col in ['customer_name', 'product_category', 'payment_method', 'status']:
        if col in transformed_df.columns and transformed_df[col].dtype == 'object':
            transformed_df[col] = transformed_df[col].str.strip()

    # Standardizing 'status' casing
    if 'status' in transformed_df.columns:
        transformed_df['status'] = transformed_df['status'].str.title()

    # Handling missing values in critical columns
    transformed_df.dropna(subset=['amount', 'quantity', 'transaction_date', 'transaction_timestamp'], inplace=True)

    # Data Type Conversion
    transformed_df['transaction_date'] = pd.to_datetime(transformed_df['transaction_date'], errors='coerce')
    transformed_df['transaction_timestamp'] = pd.to_datetime(transformed_df['transaction_timestamp'], errors='coerce')
    transformed_df['last_updated_timestamp'] = pd.to_datetime(transformed_df['last_updated_timestamp'], errors='coerce')
    transformed_df['amount'] = pd.to_numeric(transformed_df['amount'], errors='coerce')
    transformed_df['quantity'] = pd.to_numeric(transformed_df['quantity'], errors='coerce')

    # Calculating total_price
    transformed_df['total_price'] = transformed_df['quantity'] * transformed_df['amount']

    # Column Selection and Renaming
    selected_cols = [
        'record_id',
        'transaction_date',
        'transaction_timestamp',
        'customer_name',
        'product_category',
        'quantity',
        'amount',
        'total_price',
        'payment_method',
        'status'
    ]
    transformed_df = transformed_df[[col for col in selected_cols if col in transformed_df.columns]]

    transformed_df.rename(columns={
        'record_id': 'transaction_id',
        'customer_name': 'customer',
        'product_category': 'category',
        'payment_method': 'payment_type'
    }, inplace=True)

    return transformed_df

#### Transform full data

In [25]:
print("\n--- Transforming Full Data ---")
if not full_data_df.empty:
    transformed_full_df = apply_transformations(full_data_df)
    print(f"Transformed {len(transformed_full_df)} records for full data.")
    print("First 5 rows of transformed full data:\n", transformed_full_df.head())
    print("\nData Types:\n", transformed_full_df.info())

    try:
        transformed_full_df.to_csv(OUTPUT_FULL_TRANSFORMED_FILENAME, index=False)
        print(f"Transformed full data saved to '{OUTPUT_FULL_TRANSFORMED_FILENAME}'")
    except Exception as e:
        print(f"Error saving transformed full data: {e}")
else:
    print("Full data is empty, skipping transformation.")


--- Transforming Full Data ---
Transformed 398 records for full data.
First 5 rows of transformed full data:
    transaction_id transaction_date transaction_timestamp customer  \
0               1       2025-04-01   2025-04-01 17:49:57   Costco   
1               2       2025-04-01   2025-04-01 10:36:18    Apple   
2               3       2025-04-01   2025-04-01 17:16:47   Target   
3               4       2025-04-01   2025-04-01 18:15:48   Costco   
4               5       2025-04-01   2025-04-01 02:37:35   Amazon   

      category  quantity   amount  total_price payment_type     status  
0  Electronics         1   929.94       929.94       PayPal    Pending  
1    Groceries         4   130.17       520.68       PayPal  Cancelled  
2   Home Goods         8  1333.76     10670.08  Credit Card    Pending  
3        Tools         9  1115.54     10039.86       PayPal  Cancelled  
4     Software         6   633.14      3798.84  Credit Card  Cancelled  
<class 'pandas.core.frame.DataFrame'

#### Incremental Data

In [26]:
if not incremental_data_df.empty:
    transformed_incremental_df = apply_transformations(incremental_data_df)
    print(f"Transformed {len(transformed_incremental_df)} records for incremental data.")
    print("First 5 rows of transformed incremental data:\n", transformed_incremental_df.head())
    print("\nData Types:\n", transformed_incremental_df.info())

    try:
        transformed_incremental_df.to_csv(OUTPUT_INCREMENTAL_TRANSFORMED_FILENAME, index=False)
        print(f"Transformed incremental data saved to '{OUTPUT_INCREMENTAL_TRANSFORMED_FILENAME}'")
    except Exception as e:
        print(f"Error saving transformed incremental data: {e}")
else:
    print("Incremental data is empty, skipping transformation.")

Transformed 33 records for incremental data.
First 5 rows of transformed incremental data:
      transaction_id transaction_date transaction_timestamp customer  \
352             353       2025-06-15   2025-06-15 04:08:57   Costco   
353             354       2025-06-16   2025-06-16 20:11:05   Costco   
354             355       2025-06-17   2025-06-17 03:46:57   Amazon   
355             356       2025-06-18   2025-06-18 13:03:27   Amazon   
356             357       2025-06-19   2025-06-19 15:37:21   Google   

       category  quantity   amount  total_price   payment_type     status  
352       Books         5  1621.91      8109.55  Bank Transfer    Pending  
353       Books         2   432.92       865.84     Debit Card  Cancelled  
354       Tools         7   876.54      6135.78     Debit Card  Completed  
355       Books         5  1435.31      7176.55    Credit Card  Completed  
356  Home Goods         3    39.00       117.00  Bank Transfer    Pending  
<class 'pandas.core.frame

### Update Last Extraction

In [27]:
print(f"\n--- Saving New Timestamp ---")
current_extraction_time = datetime.now()
try:
    with open(LAST_EXTRACTION_FILE, 'w') as f:
        f.write(current_extraction_time.isoformat())
    print(f"New extraction timestamp saved: {current_extraction_time.isoformat()}")
except Exception as e:
    print(f"Error saving new timestamp to '{LAST_EXTRACTION_FILE}': {e}")


--- Saving New Timestamp ---
New extraction timestamp saved: 2025-06-16T09:32:26.151760
